# ImpPres Baseline

This notebook illustrates how to use the DeBERTa-v3-base-mnli-fever-anli model to perform specialized inference on the ImpPres dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [2]:
label_names = ["entailment", "neutral", "contradiction"]
def evaluate(premise, hypothesis):
    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].to(device))
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
    return prediction

In [3]:
# returns a dictionary with the scores for each label (entailment, neutral, contradiction)
evaluate("The weather is nice today.", "It is sunny outside.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entailment': 0.1, 'neutral': 99.8, 'contradiction': 0.0}

In [4]:
# get the highest score from the prediction dict (evaluate method)  
def get_prediction(pred_dict):
    if pred_dict["entailment"] > pred_dict["contradiction"]  and pred_dict["entailment"] > pred_dict["neutral"]:
        return "entailment"
    elif pred_dict["contradiction"] > pred_dict["entailment"]:
        return "contradiction"
    else:
        return "neutral"

## Load ImpPres Dataset

In [5]:
from datasets import load_dataset

sections = ['presupposition_all_n_presupposition', 
            'presupposition_both_presupposition', 
            'presupposition_change_of_state', 
            'presupposition_cleft_existence', 
            'presupposition_cleft_uniqueness', 
            'presupposition_only_presupposition', 
            'presupposition_possessed_definites_existence', 
            'presupposition_possessed_definites_uniqueness', 
            'presupposition_question_presupposition']

dataset = {}
for section in sections:
    print(f"Loading dataset for section: {section}")
    dataset[section] = load_dataset("facebook/imppres", section)


Loading dataset for section: presupposition_all_n_presupposition


README.md: 0.00B [00:00, ?B/s]

c:\Users\wexle\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wexle\.cache\huggingface\hub\datasets--facebook--imppres. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

(…)_n_presupposition-00000-of-00001.parquet:   0%|          | 0.00/43.0k [00:00<?, ?B/s]

Generating all_n_presupposition split:   0%|          | 0/1900 [00:00<?, ? examples/s]

...

Generating possessed_definites_uniqueness split:   0%|          | 0/1900 [00:00<?, ? examples/s]

Loading dataset for section: presupposition_question_presupposition


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)on_presupposition-00000-of-00001.parquet:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

Generating question_presupposition split:   0%|          | 0/1900 [00:00<?, ? examples/s]

In [6]:
dataset

{'presupposition_all_n_presupposition': DatasetDict({
     all_n_presupposition: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_both_presupposition': DatasetDict({
     both_presupposition: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_change_of_state': DatasetDict({
     change_of_state: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_cleft_existence': DatasetDict({
     cleft_existence: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UI

In [7]:
# Evaluate the model on the ImpPres dataset
from tqdm import tqdm
def evaluate_on_dataset(dataset):
    results = []
    label_names = ["entailment", "neutral", "contradiction"]
    for example in tqdm(dataset):
        premise = example['premise']
        hypothesis = example['hypothesis']
        prediction = evaluate(premise, hypothesis)
        results.append({
            'premise': premise,
            'hypothesis': hypothesis,
            'prediction': prediction,
            'pred_label': get_prediction(prediction),
            'gold_label': label_names[example['gold_label']],
        })
    return results

## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [8]:
import evaluate as eval_lib

accuracy = eval_lib.load("accuracy")
precision = eval_lib.load("precision")
recall = eval_lib.load("recall")
f1 = eval_lib.load("f1")


In [9]:
clf_metrics = eval_lib.combine(["accuracy", "f1", "precision", "recall"])

In [10]:
clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

{'accuracy': 0.6666666666666666,
 'f1': 0.6666666666666666,
 'precision': 1.0,
 'recall': 0.5}

## Your Turn

Compute the classification metrics on the baseline model on each section of the ImpPres dataset.

https://www.kaggle.com/code/faijanahamadkhan/llm-evaluation-framework-hugging-face provides good documentation on how to use the Huggingface evaluate library.

## Task 2.2

In [11]:
def compute_metrics_on_section(dataset_section, section_name):
    """
    Compute classification metrics for a specific presupposition section
    """
    # Get the actual split name for this section
    split_name = list(dataset_section.keys())[0]
    section_data = dataset_section[split_name]
    
    print(f"\n Evaluating on {section_name}...")
    print(f"Number of samples: {len(section_data)}")
    
    # Run evaluation on the section
    predictions = evaluate_on_dataset(section_data)
    
    # Extract predictions and gold labels
    pred_labels = [p['pred_label'] for p in predictions]
    gold_labels = [p['gold_label'] for p in predictions]
    
    # Map labels to integers for metrics computation
    label_to_int = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
    pred_ints = [label_to_int[label] for label in pred_labels]
    gold_ints = [label_to_int[label] for label in gold_labels]
    
    # Compute metrics using evaluate package with macro averaging for multiclass
    accuracy_result = accuracy.compute(predictions=pred_ints, references=gold_ints)
    f1_result = f1.compute(predictions=pred_ints, references=gold_ints, average='macro')
    precision_result = precision.compute(predictions=pred_ints, references=gold_ints, average='macro')
    recall_result = recall.compute(predictions=pred_ints, references=gold_ints, average='macro')
    
    metrics = {
        'accuracy': accuracy_result['accuracy'],
        'f1': f1_result['f1'],
        'precision': precision_result['precision'],
        'recall': recall_result['recall']
    }
    
    # Add section information
    metrics['section'] = section_name
    metrics['num_samples'] = len(predictions)
    
    print(f" Results for {section_name}:")
    print(f"- Samples: {metrics['num_samples']}")
    print(f"- Accuracy: {metrics['accuracy']:.3f}")
    print(f"- F1 (macro): {metrics['f1']:.3f}")
    print(f"- Precision (macro): {metrics['precision']:.3f}")
    print(f"- Recall (macro): {metrics['recall']:.3f}")
    
    return metrics, predictions

# Evaluate each presupposition section
all_results = {}
all_predictions = []

for section in sections:
    metrics, predictions = compute_metrics_on_section(dataset[section], section)
    all_results[section] = metrics
    all_predictions.extend(predictions)

print("\n" + "=" * 80)
print(" COMPUTING OVERALL METRICS ACROSS ALL SECTIONS")
print("=" * 80)



 Evaluating on presupposition_all_n_presupposition...
Number of samples: 1900


100%|██████████| 1900/1900 [06:05<00:00,  5.20it/s]


 Results for presupposition_all_n_presupposition:
- Samples: 1900
- Accuracy: 0.463
- F1 (macro): 0.420
- Precision (macro): 0.433
- Recall (macro): 0.475

 Evaluating on presupposition_both_presupposition...
Number of samples: 1900


100%|██████████| 1900/1900 [07:12<00:00,  4.39it/s]


 Results for presupposition_both_presupposition:
- Samples: 1900
- Accuracy: 0.397
- F1 (macro): 0.316
- Precision (macro): 0.275
- Recall (macro): 0.394

 Evaluating on presupposition_change_of_state...
Number of samples: 1900


...

 Results for presupposition_possessed_definites_existence:
- Samples: 1900
- Accuracy: 0.671
- F1 (macro): 0.664
- Precision (macro): 0.799
- Recall (macro): 0.737

 Evaluating on presupposition_possessed_definites_uniqueness...
Number of samples: 1900


100%|██████████| 1900/1900 [05:57<00:00,  5.31it/s]


 Results for presupposition_possessed_definites_uniqueness:
- Samples: 1900
- Accuracy: 0.390
- F1 (macro): 0.293
- Precision (macro): 0.252
- Recall (macro): 0.383

 Evaluating on presupposition_question_presupposition...
Number of samples: 1900


100%|██████████| 1900/1900 [05:46<00:00,  5.49it/s]


 Results for presupposition_question_presupposition:
- Samples: 1900
- Accuracy: 0.625
- F1 (macro): 0.599
- Precision (macro): 0.699
- Recall (macro): 0.695

 COMPUTING OVERALL METRICS ACROSS ALL SECTIONS


In [12]:
# Compute overall metrics across all sections
print(f"Computing overall metrics on {len(all_predictions)} total samples...")

# Extract overall predictions and gold labels
overall_pred_labels = [p['pred_label'] for p in all_predictions]
overall_gold_labels = [p['gold_label'] for p in all_predictions]

# Map labels to integers
label_to_int = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
overall_pred_ints = [label_to_int[label] for label in overall_pred_labels]
overall_gold_ints = [label_to_int[label] for label in overall_gold_labels]

# Compute overall metrics with macro averaging for multiclass
overall_accuracy_result = accuracy.compute(predictions=overall_pred_ints, references=overall_gold_ints)
overall_f1_result = f1.compute(predictions=overall_pred_ints, references=overall_gold_ints, average='macro')
overall_precision_result = precision.compute(predictions=overall_pred_ints, references=overall_gold_ints, average='macro')
overall_recall_result = recall.compute(predictions=overall_pred_ints, references=overall_gold_ints, average='macro')

overall_metrics = {
    'accuracy': overall_accuracy_result['accuracy'],
    'f1': overall_f1_result['f1'],
    'precision': overall_precision_result['precision'],
    'recall': overall_recall_result['recall']
}
overall_metrics['section'] = 'ALL_SECTIONS'
overall_metrics['num_samples'] = len(all_predictions)

print(f"  Overall Results across all presupposition sections:")
print(f"- Total Samples: {overall_metrics['num_samples']}")
print(f"- Accuracy: {overall_metrics['accuracy']:.3f}")
print(f"- F1 (macro): {overall_metrics['f1']:.3f}")
print(f"- Precision (macro): {overall_metrics['precision']:.3f}")
print(f"- Recall (macro): {overall_metrics['recall']:.3f}")

# Add overall results to the results dictionary
all_results['ALL_SECTIONS'] = overall_metrics


print("\n SUMMARY TABLE: DeBERTA BASELINE PERFORMANCE ON IMPPRES PRESUPPOSITIONS")
print("=" * 80)

# Create results table
print(f"{'Section':<45} {'Samples':<8} {'Accuracy':<9} {'F1':<6} {'Precision':<10} {'Recall':<6}")
print("-" * 90)

# Print individual section results
for section in sections:
    r = all_results[section]
    print(f"{section:<45} {r['num_samples']:<8} {r['accuracy']:<9.3f} {r['f1']:<6.3f} {r['precision']:<10.3f} {r['recall']:<6.3f}")

# Print overall results
print("-" * 90)
r = all_results['ALL_SECTIONS']
print(f"{'ALL_SECTIONS':<45} {r['num_samples']:<8} {r['accuracy']:<9.3f} {r['f1']:<6.3f} {r['precision']:<10.3f} {r['recall']:<6.3f}")



Computing overall metrics on 17100 total samples...
  Overall Results across all presupposition sections:
- Total Samples: 17100
- Accuracy: 0.475
- F1 (macro): 0.468
- Precision (macro): 0.493
- Recall (macro): 0.503

 SUMMARY TABLE: DeBERTA BASELINE PERFORMANCE ON IMPPRES PRESUPPOSITIONS
Section                                       Samples  Accuracy  F1     Precision  Recall
------------------------------------------------------------------------------------------
presupposition_all_n_presupposition           1900     0.463     0.420  0.433      0.475 
presupposition_both_presupposition            1900     0.397     0.316  0.275      0.394 
presupposition_change_of_state                1900     0.308     0.316  0.334      0.324 
presupposition_cleft_existence                1900     0.641     0.630  0.678      0.694 
presupposition_cleft_uniqueness               1900     0.195     0.191  0.214      0.186 
presupposition_only_presupposition            1900     0.583     0.565  0.646 

In [17]:
temp_metrics, temp_predictions = compute_metrics_on_section(dataset["presupposition_all_n_presupposition"], "all_n_presupposition")


 Evaluating on all_n_presupposition...
Number of samples: 1900


100%|██████████| 1900/1900 [05:44<00:00,  5.52it/s]

 Results for all_n_presupposition:
- Samples: 1900
- Accuracy: 0.463
- F1 (macro): 0.420
- Precision (macro): 0.433
- Recall (macro): 0.475


In [19]:
DeBERTa_v3_predictions = temp_predictions
%store DeBERTa_v3_predictions

Stored 'DeBERTa_v3_predictions' (list)


In [20]:
for pred in DeBERTa_v3_predictions:
    print(pred)

{'premise': 'All ten guys that proved to boast were divorcing.', 'hypothesis': 'There are exactly ten guys that proved to boast.', 'prediction': {'entailment': 69.3, 'neutral': 28.1, 'contradiction': 2.6}, 'pred_label': 'entailment', 'gold_label': 'entailment'}
{'premise': 'All ten guys that proved to boast were divorcing.', 'hypothesis': 'There are exactly eleven guys that proved to boast.', 'prediction': {'entailment': 0.6, 'neutral': 1.8, 'contradiction': 97.6}, 'pred_label': 'contradiction', 'gold_label': 'contradiction'}
{'premise': 'All ten guys that proved to boast were divorcing.', 'hypothesis': 'There are exactly ten senators that proved to boast.', 'prediction': {'entailment': 0.1, 'neutral': 99.5, 'contradiction': 0.4}, 'pred_label': 'contradiction', 'gold_label': 'neutral'}
{'premise': "All ten guys that proved to boast weren't divorcing.", 'hypothesis': 'There are exactly ten guys that proved to boast.', 'prediction': {'entailment': 70.4, 'neutral': 27.9, 'contradiction': 